In [ ]:
import pandas as pd
import uproot
import uproot_methods
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

import numpy as np

import functions_giammi as fugi
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

dir = r"../PYTHON_graphs/OUTPUTS/Ar_calibration/"
cmap_viridis = mpl.cm.get_cmap('viridis')



In [ ]:
N2 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/Ar_FULL_analysis.root")
# N2["all_toghether/"].keys()
# N2["all_toghether/"].classnames()
# N2["all_toghether/fish filet y"].all_members
# N2["all_toghether/fish filet y"].get_values()
# type(N2["all_toghether/fish filet y"])
# N2["all_toghether/tof"].values()
